In [ ]:
import pandas as pd
import numpy as np
import time
import random
import lightgbm as lgb
import time
from multiprocessing import Pool
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt

In [ ]:
from process import gen_map_dist

In [ ]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'
TEMP = OUTPUT + 'temp/'
MAP  = OUTPUT + 'map/'

In [ ]:
# original input
df_structures = pd.read_csv(FOLDER+'structures.csv')
df_train = pd.read_csv(FOLDER+'train.csv')
df_test = pd.read_csv(FOLDER+'test.csv')

In [ ]:
MAP_DATE = '20190818'

In [ ]:
# input files


In [ ]:
# output files
temp_map_train = TEMP + 'map_train_{}.pickle'
temp_dist_train = TEMP + 'dist_train_{}.pickle'
temp_map_test = TEMP + 'map_test_{}.pickle'
temp_dist_test = TEMP + 'dist_test_{}.pickle'

map_train = MAP + MAP_DATE + '_' + 'map_train.pickle'
map_test = MAP + MAP_DATE + '_' + 'map_test.pickle'

dist_train = MAP + MAP_DATE + '_' + 'dist_train.pickle'
dist_test = MAP + MAP_DATE + '_' + 'dist_test.pickle'

In [ ]:
df_train_idx = df_train.set_index('molecule_name')
df_structures_idx = df_structures.set_index('molecule_name')
df_test_idx = df_test.set_index('molecule_name')
mols_train = df_train['molecule_name'].unique()
mols_test = df_test['molecule_name'].unique()

In [ ]:
num_div = 8
num_col = 1

In [ ]:
def gen_mapping_train(temp_no):
    df_train_map = pd.DataFrame()
    df_train_dist = pd.DataFrame()

    div = len(mols_train) // num_div
    res = len(mols_train) % num_div
    if temp_no == num_div - 1:
        last_slice = div*(temp_no+1) + res
    else:
        last_slice = div*(temp_no+1)
    for m in tqdm(mols_train[div*temp_no:last_slice]):
        df_train_map_temp, df_train_dist_temp = gen_map_dist(df_train_idx, df_structures_idx, num_col, m)
        df_train_map = pd.concat([df_train_map, df_train_map_temp], axis=0)
        df_train_dist = pd.concat([df_train_dist, df_train_dist_temp], axis=0)
        if df_train_idx.loc[m].shape[0] != df_train_dist_temp.shape[0]:
            if type(df_train_idx.loc[m]) == pd.Series:
                continue
            else:
                print('unmatch shape ', m)
    
    df_train_map.to_pickle(temp_map_train.format(temp_no))
    df_train_dist.to_pickle(temp_dist_train.format(temp_no))



In [ ]:
start = time.time()
with Pool(processes=num_div) as p:
    p.map(gen_mapping_train, [i for i in range(num_div)])

elapsed_time = time.time() - start
print ("total elapsed_time:{0}".format(elapsed_time) + "[sec]")    

In [ ]:
def gen_mapping_test(temp_no)
    df_test_map = pd.DataFrame()
    df_test_dist = pd.DataFrame()

    div = len(mols_test) // num_div
    res = len(mols_test) % num_div
    if temp_no == num_div - 1:
        last_slice = div*(temp_no+1) + res
    else:
        last_slice = div*(temp_no+1)

    for m in mols_test[div*temp_no:last_slice]:
        df_test_map_temp, df_test_dist_temp = gen_map_dist(df_test_idx, df_structures_idx, num_col, m)
        df_test_map = pd.concat([df_test_map, df_test_map_temp], axis=0)
        df_test_dist = pd.concat([df_test_dist, df_test_dist_temp], axis=0)

    df_test_map.to_pickle(temp_map_test.format(temp_no))
    df_test_dist.to_pickle(temp_dist_test.format(temp_no))


In [ ]:
start = time.time()
with Pool(processes=num_div) as p:
    p.map(gen_mapping_test, [i for i in range(num_div)])

elapsed_time = time.time() - start
print ("total elapsed_time:{0}".format(elapsed_time) + "[sec]")    

In [ ]:
# div = len(mols_train) // num_div

# df_train_map = pd.DataFrame()
# df_train_dist = pd.DataFrame()

# start = time.time()

# # for m in mols_train[:10]:
# for m in mols_train[div*temp_no:]:
#     df_train_map_temp, df_train_dist_temp = gen_map_dist(df_train_idx, df_structures_idx, m)
#     df_train_map = pd.concat([df_train_map, df_train_map_temp], axis=0)
#     df_train_dist = pd.concat([df_train_dist, df_train_dist_temp], axis=0)
#     if df_train_idx.loc[m].shape[0] != df_train_dist_temp.shape[0]:
#         if type(df_train_idx.loc[m]) == pd.Series:
#             continue
#         else:
#             print('unmatch shape ', m)
    
# # df_train_map.to_pickle(temp_map_train.format(temp_no))
# # df_train_dist.to_pickle(temp_dist_train.format(temp_no))

# elapsed_time = time.time() - start
# print ("total elapsed_time:{0}".format(elapsed_time) + "[sec]")

In [ ]:
# div = len(mols_test) // num_div

# df_test_map = pd.DataFrame()
# df_test_dist = pd.DataFrame()

# start = time.time()

# # for m in mols_test[:10]:
# for m in mols_test[div*temp_no:]:
#     df_test_map_temp, df_test_dist_temp = gen_map_dist(df_test_idx, df_structures_idx, m)
#     df_test_map = pd.concat([df_test_map, df_test_map_temp], axis=0)
#     df_test_dist = pd.concat([df_test_dist, df_test_dist_temp], axis=0)
    
# df_test_map.to_pickle(temp_map_test.format(temp_no))
# df_test_dist.to_pickle(temp_dist_test.format(temp_no))

# elapsed_time = time.time() - start
# print ("total elapsed_time:{0}".format(elapsed_time) + "[sec]")

In [ ]:
df_map_train_list = []
df_map_test_list = []

df_dist_train_list = []
df_dist_test_list = []


for i in range(num_div):
    temp1 = pd.read_pickle(temp_map_train.format(i)).shape[0]
    temp2 = pd.read_pickle(temp_map_test.format(i)).shape[0]
    
    df_map_train_list.append(pd.read_pickle(temp_map_train.format(i)))
    df_map_test_list.append(pd.read_pickle(temp_map_test.format(i)))
    
    df_dist_train_list.append(pd.read_pickle(temp_dist_train.format(i)))
    df_dist_test_list.append(pd.read_pickle(temp_dist_test.format(i)))
    
df_map_train = pd.concat(df_map_train_list, axis=0)
df_map_test = pd.concat(df_map_test_list, axis=0)

df_dist_train = pd.concat(df_dist_train_list, axis=0)
df_dist_test = pd.concat(df_dist_test_list, axis=0)

In [ ]:
df_map_train.to_pickle(map_train)
df_map_test.to_pickle(map_test)

df_dist_train.to_pickle(dist_train)
df_dist_test.to_pickle(dist_test)

In [ ]:
print(df_map_train.shape, 
df_map_test.shape,
df_dist_train.shape,
df_dist_test.shape)